In [1]:
# 0 := delete \n
# 1 := leave as it is

# model = gensim.models.Word2Vec.load('../../../pretrained_model/kor/ko.bin')

# from hangul_utils import split_syllables, join_jamos
# import gensim

In [2]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [3]:
first_np, second_np, label_np  = data.getData()
first_np, second_np = utils.process_splitted(first_np, second_np)
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

linearized complete!
done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [4]:
pretrained_word2vec = pm.load_pretrained_model(ch2idx)
pretrained_word2vec = torch.tensor(pretrained_word2vec)

There are 27 / 133 pretrained vectors found.


In [5]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [6]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=50)

In [7]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('temp/tests')

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [8]:
# ====================
# set parameters here
# ====================

title = 'cnn-test1'
epochs = 40

vocab_size=len(ch2idx)
embed_dim = 200
hidden_size = 100
num_classes = 2
rnn_layers = 1

num_filters = [100, 100, 100]
kernel_sizes = [21, 21, 21]

dropout = 0.0

learning_rate = 0.001
weight_decay = 1e-4

model_name="CNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=pretrained_word2vec
freeze_embedding=False,

In [9]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=vocab_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing with pretrained model!!!
OCR_cnn(
  (emb): Embedding(133, 200)
  (cnn1): CNN(
    (conv1d_list): ModuleList(
      (0): Conv1d(200, 100, kernel_size=(9,), stride=(1,))
      (1): Conv1d(200, 100, kernel_size=(9,), stride=(1,))
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (cnn2): CNN(
    (conv1d_list): ModuleList(
      (0): Conv1d(200, 100, kernel_size=(9,), stride=(1,))
      (1): Conv1d(200, 100, kernel_size=(9,), stride=(1,))
    )
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (fc1): Linear(in_features=400, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.5, inplace=False)
  (dp2): Dropout(p=0.5, inplace=False)
)


In [10]:
trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.668461   | 59.732510  | 0.565014 | 69.89  |  9.12 
   2    |   0.581172   | 68.094650  | 0.524993 | 73.63  |  9.13 
   3    |   0.545897   | 71.460905  | 0.473946 | 77.07  |  9.11 
   4    |   0.517235   | 73.572016  | 0.448672 | 77.00  |  9.16 
   5    |   0.492956   | 75.123457  | 0.392489 | 81.93  |  9.07 
   6    |   0.466830   | 76.263374  | 0.403125 | 80.26  |  9.06 
   7    |   0.458219   | 76.983539  | 0.405117 | 78.11  |  9.09 
   8    |   0.454210   | 77.390947  | 0.373339 | 82.96  |  9.10 
   9    |   0.449901   | 77.641975  | 0.368865 | 84.52  |  9.12 
  10    |   0.438141   | 78.427984  | 0.378027 | 83.15  |  9.13 
  11    |   0.438951   | 78.119342  | 0.353513 | 84.22  |  9.13 
  12    |   0.433598   | 78.823045  | 0.376922 | 83.15  |  9.15 
  13    |   0.432235   | 79.000000  | 0.366821 | 81.2

KeyboardInterrupt: 

In [ ]:
tester.test(test_dataloader=test_dataloader,
            device=device,
            model=model,
            title=title)

In [ ]:
mu.graphModel(train_dataloader, model, writer)

In [ ]:
mu.saveModel(title, model)